진호님이 일단 실행되게끔 만들어 본것

In [2]:
# 3_21 모델학습 후 예측한 결과가 심통치않아 새로 전처리해보기로 하였다.
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np

#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt',  'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
train_text = open(train_file,'rb').read().decode(encoding='utf-8')
test_text = open(test_file,'rb').read().decode(encoding='utf-8')

# X 값 추출 = 리뷰문장 == 피쳐
x_train = [ row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t')>0]
x_train[:3]

#y값 추출  == target
y_train= np.array([ [int(row.split('\t')[2])] for row in train_text.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in test_text.split('\n')[1:] if row.count('\t')>0])


import pandas as pd

x_df = pd.DataFrame(x_train[:10000])

y_df = pd.DataFrame(y_train[:10000])

df = pd.concat([x_df, y_df], axis=1)


bad = df[df.iloc[:,1]==0]
good = df[df.iloc[:,1]==1]

test111=pd.concat([bad,good],axis=0) #라벨이 같이 들어온다.


from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt

vectorizer = TfidfVectorizer( tokenizer= Okt().morphs ) # 매개변수를 조정하여 다양한 옵션을 적용할 수 있습니다.
X = vectorizer.fit_transform(test111.iloc[:,0])
y = test111.iloc[:,1]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [3]:
import numpy as np

X_train_dense = np.expand_dims( X_train.toarray(), axis=-1 )
X_test_dense = np.expand_dims( X_test.toarray(), axis=-1 )

In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input

# 모델 구성
model = Sequential([
    Input(shape=(X_train_dense.shape[1],)),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 모델 컴파일 및 학습
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_dense.squeeze(), y_train, epochs = 10, batch_size = 32, validation_split = 0.2)

c:\Users\uk246\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


Epoch 1/10
200/200 [==============================] - 15s 68ms/step - loss: 0.5846 - accuracy: 0.7452 - val_loss: 0.4494 - val_accuracy: 0.8119
Epoch 2/10
200/200 [==============================] - 12s 59ms/step - loss: 0.2733 - accuracy: 0.9167 - val_loss: 0.3935 - val_accuracy: 0.8194
Epoch 3/10
200/200 [==============================] - 11s 54ms/step - loss: 0.1213 - accuracy: 0.9734 - val_loss: 0.4041 - val_accuracy: 0.8181
Epoch 4/10
200/200 [==============================] - 12s 58ms/step - loss: 0.0614 - accuracy: 0.9897 - val_loss: 0.4300 - val_accuracy: 0.8169
Epoch 5/10
200/200 [==============================] - 11s 56ms/step - loss: 0.0358 - accuracy: 0.9955 - val_loss: 0.4546 - val_accuracy: 0.8106
Epoch 6/10
200/200 [==============================] - 12s 62ms/step - loss: 0.0236 - accuracy: 0.9973 - val_loss: 0.4786 - val_accuracy: 0.8075
Epoch 7/10
200/200 [==============================] - 12s 61ms/step - loss: 0.0168 - accuracy: 0.9972 - val_loss: 0.4986 - val_accuracy:

In [5]:
# model.save('3_31_DNN 튜닝모델.h5')

In [7]:
test = ["비가 오니까 약속을 취소해야 되는게 너무 싫다."]

# 학습된 TF-IDF 벡터라이저로 테스트 데이터 변환
X_test_sample = vectorizer.transform(test)
X_test_sample_dense = X_test_sample.toarray()

# 모델을 사용하여 예측 수행
predictions = model.predict(X_test_sample_dense)
print(predictions)
# 예측 결과 출력 (긍정: 1, 부정: 0)
print("예측 결과:", np.round(predictions).astype(int))

1/1 [==============================] - 0s 48ms/step
[[0.01551629]]
예측 결과: [[0]]


In [27]:
import re

def get_from_txt(txt):
    f= open(txt,"r", encoding='utf-8')
    data = f.read() #읽어오기
    data2=data.split('\n') #줄별로 나눔 

    sentences=[]
    # for i in data2:
    #     if i.strip(): 
    #         if i[0]=='[':
    #             sentences.append(i)
    # sentences

    for line in data2:
        pattern = r'\[(.*?)\]\s+\[(.*?)\]\s+(.+)'
        match = re.match(pattern, line)
        if match:
            name = match.group(1)  # 첫 번째 대괄호 안의 단어 추출
            time = match.group(2)  # 두 번째 대괄호 안의 단어 추출
            content = match.group(3)  # 대괄호 뒤의 내용 추출
            # print(name, time, content)
            temp=[name,time,content]
            sentences.append(temp)
            
    return sentences

    # for i in sentences:
    #     print(i[2]) #i[0] 이름 i[1] 시간 i[2] 문장 

# if get_from_txt('sample.txt')[0][1]
# get_from_txt('sample.txt')

[['김찬란', '오후 10:01', '사진'],
 ['김찬란', '오후 9:38', 'https://youtube.com/watch?v=o1G7DWFFHso&feature=share'],
 ['김찬란', '오후 10:42', '사진'],
 ['김찬란', '오후 10:42', '사기치고다니는 한장규'],
 ['김하영', '오후 10:45', '이야'],
 ['김하영', '오후 10:46', '게임에서도 치네'],
 ['김찬란', '오후 10:46', '진짜 니말이 맞다 진짜 친구면 친구한테 민폐되기 싫어서 돈빌려달란 소리도 안한다'],
 ['김하영', '오전 12:25', '이모티콘'],
 ['김하영', '오전 12:35', '회식끝나고 집가는중'],
 ['김찬란', '오전 12:35', '음음 초밥집 회식은 어디서 하냐'],
 ['김하영', '오전 12:35', '우리 실장형네 직이 고기집해서 글로갔다'],
 ['김하영', '오전 12:36', '회식하면 맨날 돼지고기만 먹는다'],
 ['김찬란', '오전 12:36', 'ㅋㅋㅋㅋㅋㅋ케'],
 ['김하영', '오전 12:36', '이모티콘'],
 ['김찬란', '오전 12:36', '아 내수용 회식'],
 ['김찬란', '오전 12:36', '사진'],
 ['김하영', '오전 12:36', 'ㅋㅋㅋㅋ 그런거 아니고 애들이 여기가 맛있다고 여기서먹음'],
 ['김찬란', '오전 12:37', '고기 얘시하니까'],
 ['김찬란', '오전 12:37', '나도 고기가 먹고 싶군'],
 ['김하영', '오전 12:37', 'ㅋㅋㅋㅋ '],
 ['김찬란', '오전 12:37', '낼 사먹어야겠다'],
 ['김하영', '오전 12:37', '와라 엉아가 소고기 사줄께'],
 ['김찬란', '오전 12:37', '내가 사야지 씹새야 입벌려'],
 ['김찬란', '오전 12:37', '고기 들어간'],
 ['김찬란', '오전 12:38', '너는 내가 사주고 싶으니까 '],
 ['김찬란', '오전 12:38', '취직해서

In [15]:
for i in get_from_txt('sample.txt'):
    print( i )

In [17]:
re.match(pattern, line)

NameError: name 'pattern' is not defined